In [ ]:
%pip install -qU langchain chromadb huggingface_hub sentence-transformers pypdf tiktoken

In [ ]:
%pip install langchain-community

In [ ]:
%pip install langchain-huggingface

In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [12]:
loader4 = PyPDFLoader("/content/4.pdf")
docs4 = loader4.load()
print(len(docs4))
docs4[0]

21


Document(metadata={'producer': 'Adobe PDF Library 16.0.5', 'creator': 'Adobe InDesign 17.1 (Windows)', 'creationdate': '2022-07-21T14:09:01-04:00', 'author': 'U.S. Census Bureau', 'moddate': '2022-07-21T14:55:54-04:00', 'subject': 'Household Economic Studies', 'title': 'Occupation, Earnings, and Job Characeristics', 'trapped': '/False', 'source': '/content/4.pdf', 'total_pages': 21, 'page': 0, 'page_label': '1'}, page_content='Occupation, Earnings, and Job \nCharacteristics\nJuly 2022\nP70-178\nClayton Gumber and Briana Sullivan\nCurrent Population Reports\nINTRODUCTION\nWork is a critical component of our lives and provides \na way to obtain material and nonmonetary benefits \nlike employer-provided health insurance. Scholars \nsuggest that our identities are also tied to the notion \nof “what we do” (Christiansen, 1999), and that who \nwe are is determined partly by our occupational iden -\ntity (Skorikov and Vondracek, 2011). However, work \nis time consuming—the American Time Use S

In [10]:
loader1 = PyPDFLoader("/content/1.pdf")
docs1 = loader1.load()
print(len(docs1))
docs1[0]

18


Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': '/content/1.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nal

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
chunks1 = splitter.split_documents(docs1)
chunks4 = splitter.split_documents(docs4)
len(chunks1), len(chunks4)

(161, 215)

In [18]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings, HuggingFaceEndpoint

In [19]:
embeddingModel = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
# HF KEY

In [26]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFacePipeline
llm = HuggingFaceEndpoint(
    repo_id = "Qwen/Qwen3-4B-Instruct-2507",
    task = "text-generation",
)
model = ChatHuggingFace(llm = llm)


In [27]:
from langchain_community.vectorstores import Chroma
import chromadb

In [28]:
os.getcwd()

'/content'

In [29]:
CURRENT_DIR = os.path.dirname(os.path.abspath("."))
print(CURRENT_DIR)

/


In [30]:
DATABASE_DIR = os.path.join(CURRENT_DIR, "/content/db")
print(DATABASE_DIR)

/content/db


In [34]:
client_settings = chromadb.config.Settings(
    is_persistent = True,
    persist_directory = DATABASE_DIR,
    anonymized_telemetry = False
)

In [39]:
vectoreStore1 = Chroma.from_documents(
    documents = chunks1,
    embedding = embeddingModel,
    collection_name = "pdf1",
    client_settings = client_settings,
    collection_metadata={"hnsw": "cosine"},
    persist_directory = "/store/pdf1"
)

In [38]:
vectoreStore4 = Chroma.from_documents(
    documents = chunks4,
    embedding = embeddingModel,
    collection_name = "pdf4",
    client_settings = client_settings,
    collection_metadata={"hnsw": "cosine"},
    persist_directory = "/store/pdf4"
)

In [40]:
retriever1 = vectoreStore1.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})
retriever4 = vectoreStore4.as_retriever(search_type="mmr",search_kwargs={"k": 5, "include_metadata": True})


### Method 1) Merging Different Retrivers for using them as MergerRetriver

In [ ]:
%pip install langchain-classic

In [44]:
from langchain_classic.retrievers import MergerRetriever
mergerRetriever  = MergerRetriever(retrievers = [retriever1, retriever4],weights = [0.5, 0.5])

In [51]:
quesPdf1 = "WHAT ARE THE DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"
retrivedChunks1 = mergerRetriever.invoke(quesPdf1)
len(retrivedChunks1)
for i in retrivedChunks1:
  print(i.page_content)

sum to more than the total number with any 
coverage.
• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured
ance. According to the Bureau 
of Labor Statistics (2018), health 
insurance made up 8.3 percent 
of employers’ costs for the total 
compensation awarded to civilian 
workers in December 2017.
Given the overall cost of health 
insurance for employers, and the 
importance of health insurance 
to workers, this report explores 
workers’ private health insurance 
as well as their source of cover-
age. In contrast with other sur-
veys, the SIPP collects detailed 
information on individuals’ health
the highest rate of priva

In [65]:
quesPdf4 = "WHAT ARE THE EARNINGS AND EMPLOYEE BENEFITS?"
retrivedChunks4 = mergerRetriever.invoke(quesPdf4)
len(retrivedChunks4)
for i in retrivedChunks4:
  print(i.page_content)

state-indicator/marketplace-enrollment/?cu
rrentTimeframe=0&sortModel=%7B%22colId
%22:%22Location%22,%22sort%22:%22asc%2
2%7D>, and <www.healthaffairs.org/content/
forefront/new-biden-executive-order-aims-
build-coverage-gains>.
24 U.S. Department of Labor, Bureau of 
Labor Statistics, Local Area Unemployment 
Statistics, Unemployment Rates for States, 
2022 Annual Averages, <www.bls.gov/lau/
lastrk22.htm>, and Unemployment Rates for 
States, 2021 Annual Averages, <www.bls.gov/
share of their workweek working 
from home (81.9 percent relative 
to 88.1 percent).  
EARNINGS AND EMPLOYEE 
BENEFITS 
Monetary compensation is 
another important factor related 
to job quality, given the link 
between workers’ earnings and 
their standard of living. Earnings, 
as collected in the SIPP, refer 
to all monetary compensation 
from a work arrangement and 
include wages and salaries, 25 
25 Workers earning wages or salaries 
can report their income in several ways. For
Employment-
based
Direct  
pur

### Method 2) Using Hybrid Search Retriver(Dense+Sparse) as merger retriever

In [ ]:
%pip install rank_bm25

In [55]:
mergedChunks = chunks1 + chunks4
print(len(mergedChunks))
print(mergedChunks[0])

376
page_content='Health Insurance Coverage Status and Type 
by Geography: 2021 and 2022
American Community Survey Briefs
ACSBR-015
Issued September 2023
Douglas Conway and Breauna Branch
INTRODUCTION
Demographic shifts as well as economic and govern-
ment policy changes can affect people’s access to 
health coverage. For example, between 2021 and 2022, 
the labor market continued to improve, which may 
have affected private coverage in the United States 
during that time.
1 Public policy changes included' metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': '/content/1.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}


In [59]:
from langchain_community.vectorstores import Chroma
denseVectorStore = Chroma.from_documents(mergedChunks, embeddingModel)
similarityRetriver = denseVectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [60]:
from langchain_community.retrievers import BM25Retriever
keywordRetriever = BM25Retriever.from_documents(mergedChunks)
keywordRetriever.k = 5

In [61]:
def hybridRetriever(
    query,
    denseRetriever,
    sparseRetriever,
    denseWeight=0.5,
    sparseWeight=0.5,
    rrf_k=60
):
    scores = {}

    # Dense retrieval
    denseDocs = denseRetriever.invoke(query)
    for rank, doc in enumerate(denseDocs):
        key = doc.page_content
        scores[key] = scores.get(key, 0) + denseWeight / (rank + 1 + rrf_k)

    # Sparse retrieval
    sparse_docs = sparseRetriever.invoke(query)
    for rank, doc in enumerate(sparse_docs):
        key = doc.page_content
        scores[key] = scores.get(key, 0) + sparseWeight / (rank + 1 + rrf_k)

    # Sort by final score
    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    return [doc for doc, _ in ranked_docs]


In [62]:
quesPdf1 = "WHAT ARE THE DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"
results1 = hybridRetriever(
    query=quesPdf1,
    denseRetriever=similarityRetriver,
    sparseRetriever=keywordRetriever,
    denseWeight=0.5,
    sparseWeight=0.5
)

for i, r in enumerate(results1):
    print(f"{i+1}. {r[:100]}")


1. sum to more than the total number with any 
coverage.
• From 2021 to 2022, nine states 
reported inc
2. were Florida, Kansas, Mississippi, 
North Carolina, Ohio, South 
Carolina, and Texas.
For seven stat
3. than nonexpansion states, at 34.3 
percent (Appendix Table B-5). 
Medicaid coverage accounted 
for a
4. COVERAGE BY STATE IN 2022
Private coverage may be held 
through an individual’s or family 
member’s 
5. rates above the national average, 
nine were in the South, ranging 
12 Nonexpansion states are state
6. 8 U.S. Census Bureau
which was the state’s uninsured 
rate in 2019, the year before it 
expanded Med
7. rate of 65.6 percent (Appendix 
Table B-5).
PUBLIC HEALTH INSURANCE 
COVERAGE BY STATE IN 2022
Publi


In [67]:
quesPdf4 = "WHAT ARE THE EARNINGS AND EMPLOYEE BENEFITS?"
results4 = hybridRetriever(
    query=quesPdf4,
    denseRetriever=similarityRetriver,
    sparseRetriever=keywordRetriever,
    denseWeight=0.5,
    sparseWeight=0.5
)

for i, r in enumerate(results4):
    print(f"{i+1}. {r[:100]}")

1. share of their workweek working 
from home (81.9 percent relative 
to 88.1 percent).  
EARNINGS AND 
2. This includes wages and salaries, commissions, tips, overtime, and 
bonus pay, as well as any profit
3. 2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of
4. 2 U.S. Census Bureau
ABOUT THE DATA
The Survey of Income and Program Participation
The Survey of Inc
5. than nonexpansion states, at 34.3 
percent (Appendix Table B-5). 
Medicaid coverage accounted 
for a
6. ACS collects information for the 
preceding 12 months on wages, 
salary, commissions, bonuses, or 
t
7. sum to more than the total number with any 
coverage.
• From 2021 to 2022, nine states 
reported inc


### Merger Retriever + long context reorder

In [71]:
from langchain_community.document_transformers import LongContextReorder
reorder = LongContextReorder()
mergerDocs = mergerRetriever.invoke(quesPdf1)
mergerDocsReordered = reorder.transform_documents(mergerDocs)

mergerDocsReordered

[Document(metadata={'trapped': '/False', 'moddate': '2022-07-21T14:55:54-04:00', 'title': 'Occupation, Earnings, and Job Characeristics', 'producer': 'Adobe PDF Library 16.0.5', 'source': '/content/4.pdf', 'page': 15, 'creationdate': '2022-07-21T14:09:01-04:00', 'page_label': '16', 'subject': 'Household Economic Studies', 'total_pages': 21, 'creator': 'Adobe InDesign 17.1 (Windows)', 'author': 'U.S. Census Bureau'}, page_content='ance. According to the Bureau \nof Labor Statistics (2018), health \ninsurance made up 8.3 percent \nof employers’ costs for the total \ncompensation awarded to civilian \nworkers in December 2017.\nGiven the overall cost of health \ninsurance for employers, and the \nimportance of health insurance \nto workers, this report explores \nworkers’ private health insurance \nas well as their source of cover-\nage. In contrast with other sur-\nveys, the SIPP collects detailed \ninformation on individuals’ health'),
 Document(metadata={'moddate': '2022-07-21T14:55:54

### Hybrid Search + Long context reorder

In [73]:
from langchain_core.documents import Document

def hybridRetriever(
    query,
    denseRetriever,
    sparseRetriever,
    denseWeight=0.5,
    sparseWeight=0.5,
    rrf_k=60
):
    scores = {}
    doc_map = {}

    # Dense retrieval
    denseDocs = denseRetriever.invoke(query)
    for rank, doc in enumerate(denseDocs):
        key = doc.page_content
        doc_map[key] = doc
        scores[key] = scores.get(key, 0) + denseWeight / (rank + 1 + rrf_k)

    # Sparse retrieval
    sparse_docs = sparseRetriever.invoke(query)
    for rank, doc in enumerate(sparse_docs):
        key = doc.page_content
        doc_map[key] = doc
        scores[key] = scores.get(key, 0) + sparseWeight / (rank + 1 + rrf_k)

    ranked_keys = sorted(scores, key=scores.get, reverse=True)

    return [doc_map[k] for k in ranked_keys]


hybridDocs = hybridRetriever(
    query=quesPdf1,
    denseRetriever=similarityRetriver,
    sparseRetriever=keywordRetriever
)

hybridDocsReordered = reorder.transform_documents(hybridDocs)
hybridDocsReordered


[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': '/content/1.pdf', 'total_pages': 18, 'page': 1, 'page_label': '2'}, page_content='sum to more than the total number with any \ncoverage.\n• From 2021 to 2022, nine states \nreported increases in private \ncoverage, while seven reported \ndecreases (Appendix Table B-2). \nDIFFERENCES IN THE \nUNINSURED RATE BY STATE \nIN 2022\nIn 2022, uninsured rates at the \ntime of interview ranged across \nstates from a low of 2.4 percent \nin Massachusetts to a high of 16.6 \npercent in Texas, compared to the \nnational rate of 8.0 percent.10 Ten \nof the 15 states with uninsured'),
 Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator

### Augemenation + Generation

In [78]:
def buildContext(docs):
    return "\n\n---\n\n".join(doc.page_content for doc in docs)
contextMerger = buildContext(mergerDocsReordered)
contextHybrid = buildContext(hybridDocsReordered)

print(contextMerger)
print("------------")
print(contextHybrid)


ance. According to the Bureau 
of Labor Statistics (2018), health 
insurance made up 8.3 percent 
of employers’ costs for the total 
compensation awarded to civilian 
workers in December 2017.
Given the overall cost of health 
insurance for employers, and the 
importance of health insurance 
to workers, this report explores 
workers’ private health insurance 
as well as their source of cover-
age. In contrast with other sur-
veys, the SIPP collects detailed 
information on individuals’ health

---

plans provided by their employ-
er.41, 42 On the other hand, personal 
care and service occupations, 
which are among those with the 
lowest rates of policyholders and 
any employer-provided health 
insurance, are among the occupa -
tions with the lowest estimates 
of median income. The differ -
ences in take-up or availability of 
nonwage compensation, in the 
form of employer-provided health 
41 There was no statistically signifi -
cant difference between median monthly

---

U.S. Census B

In [79]:
def generateResponse(context, query):
    prompt = f"""
    Answer the question using ONLY the context below.
    If the answer is not present, say you don't know.

    Context:
    {context}

    Question:
    {query}

    Answer:
    """
    return model.invoke(prompt)


In [82]:
mergerRetriverResponse = generateResponse(contextMerger, quesPdf1)
hybridRetriverResponse = generateResponse(contextHybrid, quesPdf1)

print("Merger + LongContext:\n", mergerRetriverResponse.content)
print("\nHybrid + LongContext:\n", hybridRetriverResponse.content)


Merger + LongContext:
 In 2022, uninsured rates at the time of interview ranged across states from a low of 2.4 percent in Massachusetts to a high of 16.6 percent in Texas, compared to the national rate of 8.0 percent. Ten of the 15 states with uninsured rates above the national average were states that have not expanded Medicaid eligibility. Two of those 15 states, Oklahoma (11.7 percent) and Missouri (8.6 percent), had recently expanded Medicaid eligibility in 2022. Twenty-nine states and the District of Columbia had an uninsured rate lower than the national average. Maine was the only state whose uninsured rate increased from 5.7 percent in 2021 to 6.6 percent in 2022. The uninsured rate in 14 metropolitan areas did not statistically change between 2021 and 2022.

Hybrid + LongContext:
 In 2022, uninsured rates at the time of interview ranged across states from a low of 2.4 percent in Massachusetts to a high of 16.6 percent in Texas, compared to the national rate of 8.0 percent. Ten